In [ ]:
import os

import rmgpy.species
import rmgpy.chemkin
import autotst.species
import autotst.reaction
import autotst.calculator.gaussian

from hotbit import Hotbit
import ase.io
import glob

In [ ]:
# load the model
chemkin_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/chem_annotated.inp"
dictionary_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/species_dictionary.txt"
transport_path = "/home/harris.se/rmg/rmg_tools/uncertainty/nheptane/tran.dat"
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(
    chemkin_path,
    dictionary_path=dictionary_path,
    transport_path=transport_path
)
print (f"Loaded model with {len(species_list)} species and {len(reaction_list)} reactions")

In [ ]:
rxn_idx = 186  # R recombo
rxn_idx = 194
rxn_idx = 274 # first H abstraction

# rxn_idx = 236 another H abstraction




rmg_rxn = reaction_list[rxn_idx]
print(rmg_rxn)
print(rmg_rxn.family)

kinetics_base_dir = '/work/westgroup/harris.se/autoscience/dft/kinetics'
reaction_base_dir = os.path.join(kinetics_base_dir, f'reaction_{rxn_idx:04}')
os.makedirs(kinetics_base_dir, exist_ok=True)

In [ ]:
def get_label(rmg_rxn):
    label = ''
    for reactant in rmg_rxn.reactants:
        label += f'{reactant.smiles}+'
    label = label[:-1]
    label += '_'
    for product in rmg_rxn.products:
        label += f'{product.smiles}+'
    label = label[:-1]
    return label

In [ ]:
label = get_label(rmg_rxn)
print(label)

In [ ]:
reaction = autotst.reaction.Reaction(label=label, rmg_reaction=rmg_rxn)
print(reaction)

In [ ]:
reaction.get_labeled_reaction()

In [ ]:
transition_states = reaction.ts["reverse"]

In [ ]:
transition_states[0]

In [ ]:
my_ts = reaction.ts['reverse'][0]

cf = autotst.species.Conformer(rmg_molecule=my_ts.rmg_molecule).get_rdkit_mol()

In [ ]:
rdkit.Chem.rdDistGeom.EmbedMolecule(cf)

In [ ]:
m3 = rdkit.Chem.AddHs(cf)

In [ ]:
rdkit.Chem.AllChem.EmbedMolecule(m3,randomSeed=0xf00d)

In [ ]:
print(rdkit.Chem.MolToMolBlock(m3)) 

In [ ]:
rdkit.Chem.rdDistGeom.GetMoleculeBoundsMatrix(m3)

In [ ]:
# print(cf.GetAtoms())
cf.GetNumAtoms()

In [ ]:
dir(cf)

In [ ]:
my_ts.get_labels()

In [ ]:
import rdkit

In [ ]:
rdkit.Chem.r

In [ ]:
cf

In [ ]:
print(my_ts._rdkit_molecule)

In [ ]:
reaction.ts['reverse'][0].get_rdkit_mol()

In [ ]:
reaction.ts['reverse'][0].ase_molecule

In [ ]:
reaction.ts['reverse'][0].get_geometries()

In [ ]:
# # reactionnerate_conformers_all(ase_calculator=Hotbit())

In [ ]:
for ts in reaction.ts['forward']:
    print(ts)

In [ ]:
# overall calc
ts_dir = os.path.join(reaction_base_dir, 'ts')
# write Gaussian input files
for i, ts in enumerate(reaction.ts['forward']):
    gaussian = autotst.calculator.gaussian.Gaussian(conformer=ts)
    calc = gaussian.get_overall_calc()
    calc.label = f'shell_fwd_ts_{i:04}'

    calc.directory = ts_dir
    calc.parameters.pop('scratch')
    calc.parameters.pop('multiplicity')
    calc.parameters['mult'] = ts.rmg_molecule.multiplicity
    
    calc.write_input(ts.ase_molecule)

print("done")

In [ ]:
# read it back in
with open(calc.label + ".com", "r") as f:
    print("")
    for line in f.readlines():
        print(line[:-1])

In [ ]:
# Get more detailed results in Gaussian